In [36]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list=os.listdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list.sort()

In [37]:
file_list

['.DS_Store',
 '2019.01.02.csv',
 '2019.01.03.csv',
 '2019.01.04.csv',
 '2019.01.07.csv',
 '2019.01.08.csv',
 '2019.01.09.csv',
 '2019.01.10.csv',
 '2019.01.11.csv',
 '2019.01.14.csv',
 '2019.01.15.csv',
 '2019.01.16.csv',
 '2019.01.17.csv',
 '2019.01.18.csv',
 '2019.01.21.csv',
 '2019.01.22.csv',
 '2019.01.23.csv',
 '2019.01.24.csv',
 '2019.01.25.csv',
 '2019.01.28.csv',
 '2019.01.29.csv',
 '2019.01.30.csv',
 '2019.01.31.csv',
 '2019.02.01.csv',
 '2019.02.11.csv',
 '2019.02.12.csv',
 '2019.02.13.csv',
 '2019.02.14.csv',
 '2019.02.15.csv',
 '2019.02.18.csv',
 '2019.02.19.csv',
 '2019.02.20.csv',
 '2019.02.21.csv',
 '2019.02.22.csv',
 '2019.02.25.csv',
 '2019.02.26.csv',
 '2019.02.27.csv',
 '2019.02.28.csv',
 '2019.03.01.csv',
 '2019.03.04.csv',
 '2019.03.05.csv',
 '2019.03.06.csv',
 '2019.03.07.csv',
 '2019.03.08.csv',
 '2019.03.11.csv',
 '2019.03.12.csv',
 '2019.03.13.csv',
 '2019.03.14.csv',
 '2019.03.15.csv',
 '2019.03.18.csv',
 '2019.03.19.csv',
 '2019.03.20.csv',
 '2019.03.21.csv'

In [38]:
class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def calc_thresholds(self,array): #get the percentile values of the array
        self.threshold.append(np.percentile(array,self.percentiles))   
     
    def return_quartile(self,array): 
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)             
        return np.asarray(temp)
    
class categorise_10():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,20,30,40,50,60,70,80,90]
        
    def calc_thresholds(self,array): #get the percentile values of the array
        self.threshold.append(np.percentile(array,self.percentiles))   
     
    def return_quartile(self,array): 
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                elif i<self.threshold[0][5]:
                    temp.append(6)
                elif i<self.threshold[0][6]:
                    temp.append(7)
                elif i<self.threshold[0][7]:
                    temp.append(8)        
                elif i<self.threshold[0][8]:
                    temp.append(9)  
                else:
                    temp.append(10)                    
        return np.asarray(temp)
class categorise_new():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,25,50,75,90]
        
    def calc_thresholds(self,array): #get the percentile values of the array
        self.threshold.append(np.percentile(array,self.percentiles))   
     
    def return_quartile(self,array): 
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                else:
                    temp.append(6)                    
        return np.asarray(temp)                                

In [39]:
cat=categorise_new()
x=np.array([1,2,3,4,5,6,7,8,9,10,11,12,13])
y=np.array([1,2,3,4,5,6,7,8,9])
cat.calc_thresholds(x)
cat.return_quartile(x)

array([1, 1, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6])

In [41]:
def calc_smart_price(dataset):
    data=dataset[:]
    
    #to combat the limit up event, where price is set to 0. 
    rows=(data.loc[:,'BidPrice1']==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        data.at[rows,'BidPrice1']=data.loc[rows,'AskPrice1'] #for that row, assign ask price to it
    rows=(data.loc[:,'AskPrice1']==0) #do the same for ask price
    if (np.any(rows)):
        data.at[rows,'AskPrice1']=data.loc[rows,'BidPrice1'] 
        
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_present_vol(dataset):
    data=dataset[:]
    data['current_vol']=data.loc[:,'Volume'].diff().fillna(0)/2
    return data

def calc_future_price(dataset,time_ahead=30):
    data=dataset[:]
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,44]+timedelta(seconds=time_ahead)
        #print(data[i,44])
        j=0
        #print(current_time)
        while((i+j)<length and current_time>data[(i+j),44]):
            j+=1
        #print(i,j,(data[(i+j-1),44]))
        if (i+j)<length:
            future_price.append(data[(i+j),51]) #51 is the index for smart price            
        else:
            future_price.append(np.nan)
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)


def calc_edge(dataset):
    data=dataset.copy()
    temp=data[:,52]-data[:,51]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset):
    data=dataset[:]
    index=data[:,44]
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,44]=new_index
    return data

def calc_sma_fast(dataset,duration=1): #0.05 seconds for 5000 rows
    data=dataset[:]
    sma_values=[] 
    smart_sum=np.cumsum(data[:,51])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            sma=(smart_sum[i]-smart_sum[i-j])/(j)
            sma_values.append(sma)
        else:
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)
    sma_values=np.asarray(sma_values)
    sma_values=data[:,51][:]-sma_values
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1) 

def calc_past_vol(dataset,duration=1): #
    data=dataset[:]
    vol_values=[] 
    vol_sum=np.cumsum(data[:,52])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) #52  

def calc_past_vol_seconds(dataset,duration=10): #
    data=dataset[:]
    vol_values=[] 
    vol_sum=np.cumsum(data[:,52])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(seconds=duration)
        j=2*duration#4 rows per second
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) #52  

def process(dataset):
    data=dataset[:]
    data=calc_smart_price(data).values #51
    data=set_index(data)
    data=calc_future_price(data) #52
    data=calc_edge(data) #53
    data=calc_sma_fast(data,duration=5) #54
    return data

ma_dict={'-4':'1',
        '-3':'5',
        '-2':'15',
        '-1':'30'}    
        
'''def calc_vwap(dataset,duration=1): #to be implement
    data=dataset[:]
    for i in data[:,44]:
        last_time=i-timedelta(minutes=duration)
        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]
        high=rolling[:,51].max()
        low=rolling[:,51].min()
        avg=(rolling[-1,51]+high+low)/3
def calc_rsi(dataset)'''

'def calc_vwap(dataset,duration=1): #to be implement\n    data=dataset[:]\n    for i in data[:,44]:\n        last_time=i-timedelta(minutes=duration)\n        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]\n        high=rolling[:,51].max()\n        low=rolling[:,51].min()\n        avg=(rolling[-1,51]+high+low)/3\ndef calc_rsi(dataset)'

In [42]:
df_list=[]
name_list=[]
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
for file in file_list: #read all files and add them to file_list
    if file[-3:]=='csv': #check if file is a CSV
        name_list.append(file)
        df_list.append(process(pd.read_csv(path+file)))
        print(file,'read')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


2019.01.02.csv read
2019.01.03.csv read
2019.01.04.csv read
2019.01.07.csv read
2019.01.08.csv read
2019.01.09.csv read
2019.01.10.csv read
2019.01.11.csv read
2019.01.14.csv read
2019.01.15.csv read
2019.01.16.csv read
2019.01.17.csv read
2019.01.18.csv read
2019.01.21.csv read
2019.01.22.csv read
2019.01.23.csv read
2019.01.24.csv read
2019.01.25.csv read
2019.01.28.csv read
2019.01.29.csv read
2019.01.30.csv read
2019.01.31.csv read
2019.02.01.csv read
2019.02.11.csv read
2019.02.12.csv read
2019.02.13.csv read
2019.02.14.csv read
2019.02.15.csv read
2019.02.18.csv read
2019.02.19.csv read
2019.02.20.csv read
2019.02.21.csv read
2019.02.22.csv read
2019.02.25.csv read
2019.02.26.csv read
2019.02.27.csv read
2019.02.28.csv read
2019.03.01.csv read
2019.03.04.csv read
2019.03.05.csv read
2019.03.06.csv read
2019.03.07.csv read
2019.03.08.csv read
2019.03.11.csv read
2019.03.12.csv read
2019.03.13.csv read
2019.03.14.csv read
2019.03.15.csv read
2019.03.18.csv read
2019.03.19.csv read


In [45]:
len(df_list)

83

In [44]:
#run regressions against all 4 moving averages
df_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/'

final_df=pd.DataFrame()    

for i in range(len(df_list)): #for each 20 day rolling window
    if i%10==0:
        print('starting 10:',i)
    x=df_list[i][:,54][:].astype(float).copy() #get x,y for regression
    y=df_list[i][:,53][:].astype(float).copy()
    
    isnum=(~np.isnan(x)) & (~np.isnan(y))
    y=y[isnum]#removing all NA
    x=x[isnum]        

    reg_result={}
    reg_result['ma_time_minutes']=1
    reg_result['date']=name_list[i]
    reg_result['total_obs']=len(x)

    result=stats.linregress(x,y)

    reg_result[('slope')]=result[0]
    reg_result[('intercept')]=result[1]
    reg_result[('r_val')]=result[2]
    reg_result[('p_val')]=result[3]
    reg_result[('std_err')]=result[4]
    reg_result[('x_mean')]=np.mean(x)
    reg_result[('x_std')]=np.std(x)

    final_df=final_df.append(reg_result,ignore_index=True)                
temp=df_path+'result_1day_5minute_ma.csv'
final_df.to_csv(temp)
print('done',temp)

starting 10: 0
starting 10: 10
starting 10: 20
starting 10: 30
starting 10: 40
starting 10: 50
starting 10: 60
starting 10: 70
starting 10: 80
done /Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_1day_5minute_ma.csv


In [17]:
df_list[0][50]

array(['2019-01-02', 1546390811852136, 'TA905', 0, 5690, 5640, 1061698,
       5622, 5642, 5622, 5638, 5636, 5634, 5632, 5630, 5628, 5638, 5640,
       5642, 5644, 5646, 108, 289, 38, 89, 121, 7, 1, 24, 196, 6, 13632,
       383717340, 1066398, 0, 5630, 5630, 7562, 5532, 5976, 5404, 13090,
       6126, 20190102, datetime.datetime(1900, 1, 1, 9, 0, 12, 500000),
       20190102, 0, 32412500, 20190102, 5556, 5752, 5637.878260869566,
       5638.901960784314, 1.0236999147482493, 0.5504113041906749],
      dtype=object)

In [45]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_1day_5minute_ma.csv')
print('mean slope',data.loc[:,'slope'].mean())

mean slope -0.0011359919278516122


In [46]:
print('percentage positive:',len(data[data.loc[:,'slope']>=0])/len(data))

percentage positive: 0.4819277108433735


In [202]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_volume_1day_4split_5minute_ma.csv')
for i in range(1,5):
    col='quartile_'+str(i)+'_slope'
    row=data.loc[:,col]
    pos=row[row>=0]
    neg=row[row<0]
    perc=len(pos)/(len(pos)+len(neg))
    print('category:',i,'percentage positive:',round(perc,7),'mean slope:',round(row.mean(),7),'std dev',round(row.std(),7))

category: 1 percentage positive: 0.65625 mean slope: 0.0227922 std dev 0.0845356
category: 2 percentage positive: 0.53125 mean slope: 0.0206701 std dev 0.0509659
category: 3 percentage positive: 0.5625 mean slope: 0.0137058 std dev 0.0622627
category: 4 percentage positive: 0.390625 mean slope: -0.0166562 std dev 0.0434371
